In [1]:
import warnings
warnings.filterwarnings("ignore")
import src.dataset as ds
from src.model import LyricsMelodyModel, LyricsOnlyModel
from src.embeddings import extract_embedding_weights
import joblib

Using TensorFlow backend.


In [2]:
x, y, tokenizer, songs = joblib.load("x_y_tokenizer_songs.jblib")
embedding_matrix = extract_embedding_weights()

100%|█████████████████████████████████████████████████████| 614/614 [00:01<00:00, 357.46it/s]


In [3]:
model = LyricsMelodyModel(tokenizer, embedding_matrix, rnn_type='gru', rnn_units=50, bidirectional=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 300)       2247300     input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 300)          0           embedding_1[0][0]                
__________________________________________________________________________________________________
dropout_1 (Dropout)  

In [4]:
model.train([x,songs], y, epochs=1)

Instructions for updating:
Use tf.cast instead.
Train on 171249 samples, validate on 19028 samples
Epoch 1/1
171249/171249 [==============================] - 29s 171us/step - loss: 6.6500 - _perplexity: 196.2823 - val_loss: 6.4227 - val__perplexity: 325.3870


In [7]:
model.predict(first_word='no', song=songs[0], n_words=50)

"no a heartache i divides dried and . why mony bout doctor how baby we some about there field bust in day rest i goodbye view go bite am gegen some calling the silly es 'm higher would wherever rendezvous still cast . you whose nicky before . was searching ."